In [9]:
def countDown(x):
    while True:
        if x == 0:
            break
            
        print("CountDown ... %d" % x)
        x -= 1
        time.sleep(1)

        
p = multiprocessing.Process(target=countDown, args=(5, ))
p.start()

In [2]:
import time
import multiprocessing

# 우리가 프로세스로 만드려고 하는 함수
def process(msg, interval):
    while True:
        print("I'm working ...: %s" % msg)
        time.sleep(interval)
        
# 프로세스로 동작할 녀석이 process 함수임을 target으로 알려준다.
# 그리고 msg와 interval에 인자(파라미터)를 전달한다.
# msg = "pX", interval은 대기 시간
# 전부 부모 프로세스의 자식 프로세스들
p1 = multiprocessing.Process(target=process, args=("p1", 1, ))
p2 = multiprocessing.Process(target=process, args=("p2", 3, ))
p3 = multiprocessing.Process(target=process, args=("p3", 5, ))
p4 = multiprocessing.Process(target=process, args=("p4", 2, ))

# 실제 구동은 여기서 시작된다.
p1.start()
p2.start()
p3.start()
p4.start()

cnt = 0

# 루프를 10번 돌면서 1초마다 Main Thread를 출력한다.
while cnt < 10:
    cnt += 1
    # Main Thread를 부모 프로세스로 보면 되겠다.
    print("Main Thread ...")
    time.sleep(1)

Main Thread ...
Main Thread ...


KeyboardInterrupt: 

In [3]:
import multiprocessing

def withdraw(balance):
    for _ in range(20000):
        balance.value -= 1
        
def deposit(balance):
    for _ in range(20000):
        balance.value += 1
        
def perform_transaction():
    balance = multiprocessing.Value('i', 20000)
    
    p1 = multiprocessing.Process(target=withdraw, args=(balance, ))
    p2 = multiprocessing.Process(target=deposit, args=(balance, ))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    
    print("Final Balance = {}".format(balance.value))
    
for _ in range(10):
    perform_transaction()

Final Balance = 20000
Final Balance = 20000
Final Balance = 20000
Final Balance = 20000
Final Balance = 20000
Final Balance = 20000
Final Balance = 20000
Final Balance = 20000
Final Balance = 20000
Final Balance = 20000


In [4]:
import multiprocessing

def withdraw(balance, lock):
    for _ in range(20000):
        lock.acquire()   # lock을 획득한다.
        balance.value -= 1
        lock.release()   # lock을 해제한다.
        
def deposit(balance, lock):
    for _ in range(20000):
        lock.acquire()
        balance.value += 1
        lock.release()
        
def perform_transaction():
    balance = multiprocessing.Value('i', 20000)
    
    # 스핀락을 만들때 아래와 같이 만든다.
    lock = multiprocessing.Lock()
    
    p1 = multiprocessing.Process(target=withdraw, args=(balance, lock, ))
    p2 = multiprocessing.Process(target=deposit, args=(balance, lock, ))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    
    print("Final Balance = {}".format(balance.value))
    
for _ in range(10):
    perform_transaction()

Final Balance = 20000
Final Balance = 20000
Final Balance = 20000
Final Balance = 20000
Final Balance = 20000
Final Balance = 20000
Final Balance = 20000
Final Balance = 20000
Final Balance = 20000
Final Balance = 20000


In [5]:
import time
from multiprocessing import Pool

def f(x):
    return x * x


# Pool은 병렬화 시킬 프로세스를 몇 개 만들지 지정한다.
# processes = 4는 프로세스를 4개 만든다는 의미다.
pool = Pool(processes = 4)

# 현재 시간을 구해서 t에 저장
t = time.mktime(time.localtime())

# Async: Asynchronous(비동기 처리)를 의미한다.
# 비동기 처리와 동기 처리의 차이는 ?
# 비동기는 일단 싹 다 질러놓고 상황 대응은 다음
# 동기는 반드시 상호간의 협약(암묵적이던 명시적이던)이 필요하다.

# apply_async도 람다 방식으로
# f 에 1000000개를 연산할 수 있도록 준비한다.
# 여기서 만든 개수 / 프로세스 숫자만큼
# 처리할 데이터의 개수를 분배하게 된다.
# 그래서 아래쪽 실제 map을 활용해서 람다 처리를 한다.
res = pool.apply_async(f, (1000000, ))
print(pool.map(f, range(1000000)))
# 모든 처리가 끝났으니 걸린 시간을 t1에 저장한다.
t1 = time.mktime(time.localtime()) - t

t = time.mktime(time.localtime())

# 여기서는 단순히 그냥 for문으로 쭉 돌린다.
# 단순히 for문만 사용하면 병렬처리가 되지 않는다.
for i in range(1000000):
    print(i * i)


t2 = time.mktime(time.localtime()) - t

# 왜 성능차이가 4 배 나지 않나요 ?
# 프로세스를 4개 생성하는데 추가적인 시간이 들어가므로
# 프로세스 생성시간을 제외한 3.x 배 정도의 성능이 최대다.
print("t1 =", t1)
print("t2 =", t2)

TimeoutError: 

In [ ]:
import time
from multiprocessing import Pool

def f(x):
    return x * x
    
pool = Pool(processes = 4)

res = pool.apply_async(f, (10, ))
print(pool.map(f, range(10)))
it = pool.imap(f, range(10))
print(it.next())
print(it.next())
print(it.next())

In [ ]:
# Thread를 사용하기 위한 라이브러리
import threading

x = 0
lock = multiprocessing.Lock()

# 값을 하나씩 더한다.
def inc_glob():
    global x
    x += 1


# 스레드가 하는 일
def taskofThread():
    for _ in range(100000):
        lock.acquire()
        inc_glob()
        lock.release()


def thread_main():
    global x
    x = 0
    
    # 스레드를 생성하고 
    # 해당 스레드는 taskofThread를 실행한다.
    t1 = threading.Thread(target = taskofThread)
    t2 = threading.Thread(target = taskofThread)
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()


for i in range(10):
    thread_main()
    print("x = {1} after Iteration {0}.".format(i, x))

In [ ]:
def square(n):
    return n * n


myList = [1, 2, 3, 4, 5]
res = []

for num in myList:
    res.append(square(num))


print(res)

In [ ]:
import multiprocessing
import os

def square(n):
    print("Worker Process id for {0}: {1}".format(n, os.getpid()))
    return n * n


myList = [1, 2, 3, 4, 5]
p = multiprocessing.Pool()
res = p.map(square, myList)

print(res)

In [ ]:
import os
import numpy as np
import multiprocessing

dx = 0.000001

def calc(x):
    #print("Process id for {0}: {1}".format(x, os.getpid))
    return dx * (x ** 2)


def integralRange(start, end):
    curX = np.arange(start, end, dx)
    #print(curX)
    p = multiprocessing.Pool()
    res = p.map(calc, curX)
    area = np.sum(res)
    return area


area = integralRange(0, 3)
print("x^2의 0 ~ 3까지 정적분 결과 =", area)



[0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13
 0.14 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27
 0.28 0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41
 0.42 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55
 0.56 0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69
 0.7  0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83
 0.84 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97
 0.98 0.99 1.   1.01 1.02 1.03 1.04 1.05 1.06 1.07 1.08 1.09 1.1  1.11
 1.12 1.13 1.14 1.15 1.16 1.17 1.18 1.19 1.2  1.21 1.22 1.23 1.24 1.25
 1.26 1.27 1.28 1.29 1.3  1.31 1.32 1.33 1.34 1.35 1.36 1.37 1.38 1.39
 1.4  1.41 1.42 1.43 1.44 1.45 1.46 1.47 1.48 1.49 1.5  1.51 1.52 1.53
 1.54 1.55 1.56 1.57 1.58 1.59 1.6  1.61 1.62 1.63 1.64 1.65 1.66 1.67
 1.68 1.69 1.7  1.71 1.72 1.73 1.74 1.75 1.76 1.77 1.78 1.79 1.8  1.81
 1.82 1.83 1.84 1.85 1.86 1.87 1.88 1.89 1.9  1.91 1.92 1.93 1.94 1.95
 1.96 

In [1]:
from PIL import Image
img = Image.open("test.jpg")

img.show()